In [22]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


In [23]:
# Load datasets
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')


In [24]:
# Add a time-related feature, 'age' of the vehicle
current_year = 2024  # Assuming the current year is 2024
train_data['age'] = current_year - train_data['model_year']
test_data['age'] = current_year - test_data['model_year']

In [25]:
# Features and target variable
X_train = train_data.drop(columns=['price', 'id'])
y_train = train_data['price']
X_test = test_data.drop(columns=['id'])

In [26]:
# Function to extract numeric horsepower
def extract_hp(engine_str):
    match = re.search(r'(\d+\.\d+|\d+)HP', engine_str)
    if match:
        return float(match.group(1))
    else:
        return np.nan

# Apply function to extract horsepower
if 'engine' in X_train.columns:
    X_train['engine_hp'] = X_train['engine'].apply(extract_hp)
if 'engine' in X_test.columns:
    X_test['engine_hp'] = X_test['engine'].apply(extract_hp)

In [27]:
# Drop the original 'engine' feature
X_train.drop(columns=['engine'], inplace=True, errors='ignore')
X_test.drop(columns=['engine'], inplace=True, errors='ignore')

# Drop rows with NaN values in 'engine_hp' (if any)
X_train = X_train.dropna(subset=['engine_hp'])
y_train = y_train.loc[X_train.index]

In [29]:
# Define numerical and categorical features
numeric_features = ['model_year', 'milage', 'engine_hp', 'age']
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# Create transformers for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers into a single preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess training and testing data
X_train_processed = preprocessor.fit_transform(X_train).toarray()  # Convert to dense array
X_test_processed = preprocessor.transform(X_test).toarray()        # Convert to dense array

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train_processed.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train the model
history = model.fit(X_train_processed, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Predict on the test set
y_test_pred = model.predict(X_test_processed)

Epoch 1/100
1256/1256 [==============================] - 11s 8ms/step - loss: 4755321344.0000 - root_mean_squared_error: 68958.8359 - val_loss: 3612024064.0000 - val_root_mean_squared_error: 60100.1172
Epoch 2/100
1256/1256 [==============================] - 7s 6ms/step - loss: 4103209216.0000 - root_mean_squared_error: 64056.2969 - val_loss: 3564805632.0000 - val_root_mean_squared_error: 59705.9922
Epoch 3/100
1256/1256 [==============================] - 8s 7ms/step - loss: 4079421952.0000 - root_mean_squared_error: 63870.3516 - val_loss: 3542907392.0000 - val_root_mean_squared_error: 59522.3281
Epoch 4/100
1256/1256 [==============================] - 6s 5ms/step - loss: 4059545088.0000 - root_mean_squared_error: 63714.5586 - val_loss: 3525197568.0000 - val_root_mean_squared_error: 59373.3750
Epoch 5/100
1256/1256 [==============================] - 8s 6ms/step - loss: 4039770368.0000 - root_mean_squared_error: 63559.1875 - val_loss: 3515020032.0000 - val_root_mean_squared_error: 59287

In [30]:
# Ensure all IDs have predictions by creating a DataFrame with all IDs
submission = pd.DataFrame({
    'id': test_data['id'],
    'price': y_test_pred.flatten()
})

# Save the predictions to a CSV file in the format [id, price]
submission.to_csv('/content/ANN_prices.csv', index=False)

In [21]:
# Since we don't have true price labels for the test set, we'll calculate RMSE on a portion of the training data held out as validation set
# Split the training data into a smaller training set and a validation set
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the best model on the split training data
best_model.fit(X_train_split, y_train_split)

# Predict on the validation set
y_val_pred = best_model.predict(X_val)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {rmse:.2f}")

# Save the predictions to a CSV file in the format [id, price]
submission = pd.DataFrame({
    'id': test_data['id'],
    'price': y_test_pred
})
submission.to_csv('/content/predicted_prices_Elasticnet.csv', index=False)

Validation RMSE: 66195.88


In [30]:
print("Length of ID column in test_data:", len(test_data['id']))
print("Length of predicted prices array:", len(y_pred))


Length of ID column in test_data: 36183
Length of predicted prices array: 33577


In [19]:
# Feature importance
importances = best_model.named_steps['regressor'].feature_importances_
feature_names = numeric_features + list(best_model.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))

# Create a DataFrame for visualization
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importances = feature_importances.sort_values('importance', ascending=False)

print(feature_importances)

AttributeError: 'ElasticNet' object has no attribute 'feature_importances_'